<img src = 'fotos/logo_dani.jpeg'>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX 
import itertools
from IPython.display import clear_output
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

## Ficheros y rutas de entrada/salida 

In [2]:
dir_in = '../../datos/datos_originales'
file1_in = 'Dataset1.- DatosConsumoAlimentarioMAPAporCCAA.txt'
dir_out = '../../datos/datos_desarrollo'

## Carga de datos 

In [3]:
df_consumo = pd.read_csv(os.path.join(dir_in, file1_in), sep = '|').drop(['Unnamed: 10', 'Unnamed: 11'], axis = 1)
df_consumo.shape

(26634, 10)

In [4]:
df_consumo[(df_consumo['Año']==2020)&(df_consumo['CCAA']=='Total Nacional')&(df_consumo['Producto']=='T.HORTALIZAS FRESCAS')]

,Año,Mes,CCAA,Producto,Volumen (miles de kg),Valor (miles de €),Precio medio kg,Penetración (%),Consumo per capita,Gasto per capita
21011,2020,Enero,Total Nacional,T.HORTALIZAS FRESCAS,"215831,81","421389,55","1,95","97,02","4,87","9,51"
21929,2020,Febrero,Total Nacional,T.HORTALIZAS FRESCAS,"226022,12","430847,66","1,91","96,53","5,1","9,72"
22847,2020,Marzo,Total Nacional,T.HORTALIZAS FRESCAS,"257552,9","498665,95","1,94","97,66","5,81","11,25"
23765,2020,Abril,Total Nacional,T.HORTALIZAS FRESCAS,"303814,22","618817,73","2,04","98,26","6,86","13,97"
24683,2020,Mayo,Total Nacional,T.HORTALIZAS FRESCAS,"288906,8","565957,25","1,96","98,05","6,52","12,77"
25601,2020,Junio,Total Nacional,T.HORTALIZAS FRESCAS,"247215,64","471135,44","1,91","97,59","5,58","10,63"
26536,2020,Julio,Total Nacional,T.HORTALIZAS FRESCAS,"242399,04","448389,61","1,85",NaN,"5,46","10,11"
26560,2020,Agosto,Total Nacional,T.HORTALIZAS FRESCAS,"217270,54","409019,27","1,88",NaN,"4,90","9,23"
26584,2020,Septiembre,Total Nacional,T.HORTALIZAS FRESCAS,"244130,14","477824,67","1,96",NaN,"5,50","10,78"
26608,2020,Octubre,Total Nacional,T.HORTALIZAS FRESCAS,"251090,91","496897,93","1,98",NaN,"5,66","11,21"


In [5]:
df_consumo.columns = df_consumo.columns.str.lower().str.replace('[\()]', '', regex = True).str.replace(' ', '_')
df_consumo.iloc[:, 4:] = df_consumo.iloc[:, 4:].apply(lambda x: x.str.replace(',','.'))
df_consumo.iloc[:, 4:] = df_consumo.iloc[:, 4:].astype(float)

df_consumo[['producto', 'ccaa']] = df_consumo[['producto', 'ccaa']].apply(lambda x: x\
                                                                          .str.lower().str.replace('+', '', regex = False)\
                                                                          .str.replace('total', '', regex = False)\
                                                                          .str.replace('.', ' ', regex = False)\
                                                                          .str.lstrip('t')\
                                                                          .str.lstrip(' ').str.capitalize())

In [ ]:
month_num = {'Enero': '01', 'Febrero': '02', 'Marzo': '03', 'Abril': '04', 'Mayo': '05', 'Junio': '06', 'Julio': '07', 'Agosto': '08',
             'Septiembre': '09', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'}
df_consumo['mes'].replace(month_num, inplace = True)

df_consumo['date'] = df_consumo['año'].astype(str) + '-' + df_consumo['mes'].astype(str) + '-01'
df_consumo.date = pd.to_datetime(df_consumo.date, format = '%Y-%m-%d')

### Asignación de clases

In [ ]:
df_consumo[(df_consumo['date']>'2020-01-01')&(df_consumo['ccaa']=='Nacional')&(df_consumo['producto']=='Hortalizas frescas')];

In [ ]:
df_consumo.producto.unique()

In [ ]:
df_consumo.date.min()

### Creación de columnas 

In [ ]:
conditions = [
    df_consumo['date'].between("2020-03-01", "2020-06-01"),
    df_consumo['date'].between("2020-06-02", "2020-09-01"),
    df_consumo['date'].between("2020-09-02", "2020-12-10"),
    (df_consumo['date'] > "2020-12-10")
    ]

values = ['Primera ola', 'Periodo de recuperación', 'Segunda ola', 'Tercera ola']
values2 = ['Confinamiento', 'Nueva normalidad', 'Vuelta al trabajo', 'Navidad']
df_consumo['Olas'] = np.select(conditions, values, default = 'Pre-covid')
df_consumo['Motivo_ola'] = np.select(conditions, values2, default = 'Pre-covid')

In [ ]:
df_consumo.to_csv(os.path.join(dir_out, 'consumo_preproc.csv'), sep = ';', index = False)